Wes McKinney freely says that his book is about data wrangling with Python, but he wants to give an introduction to `statsmodels` and `scikitlearn`, as well as ease the transition between pandas and modeling.

In [1]:
import pandas as pd
import numpy as np

## Interfacing between pandas and model code

Turn a `DataFrame` or `Series` into a numpy array by using the `values` attribute.

In [2]:
data = pd.DataFrame({
    'x0': [1, 2, 3, 4, 5],
    'x1': [0.01, -0.01, 2.5, -4.1, 0.],
    'y': [-1.5, 0., 3.6, 1.3, -2.],
})
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,2.50,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [3]:
data.values

array([[ 1.  ,  0.01, -1.5 ],
       [ 2.  , -0.01,  0.  ],
       [ 3.  ,  2.5 ,  3.6 ],
       [ 4.  , -4.1 ,  1.3 ],
       [ 5.  ,  0.  , -2.  ]])

In [4]:
data['x0'].values

array([1, 2, 3, 4, 5])

Go back the other way and turn a 2-d ndarray into a dataframe by passing column names.

In [5]:
df2 = pd.DataFrame(data.values, columns=['x0', 'x1', 'y'])
df2

,x0,x1,y
0,1.0,0.01,-1.5
1,2.0,-0.01,0.0
2,3.0,2.50,3.6
3,4.0,-4.10,1.3
4,5.0,0.00,-2.0


If you have heterogeneous data, you may need to pull out the columns that are numeric first, or you will get an ndarray of objects instead.

In [6]:
df3 = data.copy()
df3['strings'] = ['a', 'b', 'c', 'd', 'e']
df3

,x0,x1,y,strings
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,2.50,3.6,c
3,4,-4.10,1.3,d
4,5,0.00,-2.0,e


In [7]:
df3.values

array([[1, 0.01, -1.5, 'a'],
       [2, -0.01, 0.0, 'b'],
       [3, 2.5, 3.6, 'c'],
       [4, -4.1, 1.3, 'd'],
       [5, 0.0, -2.0, 'e']], dtype=object)

In [8]:
df3.loc[:, ['x0', 'x1']].values

array([[ 1.  ,  0.01],
       [ 2.  , -0.01],
       [ 3.  ,  2.5 ],
       [ 4.  , -4.1 ],
       [ 5.  ,  0.  ]])

Suppose you have a column of categorical data - you may need/want to add dummy variables for each category for a statistical of ML model using `get_dummies`.

In [9]:
data['category'] = pd.Categorical(['a', 'b', 'a', 'a', 'b'])
data

,x0,x1,y,category
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,2.50,3.6,a
3,4,-4.10,1.3,a
4,5,0.00,-2.0,b


In [10]:
dummies = pd.get_dummies(data['category'])
dummies

,a,b
0,1,0
1,0,1
2,1,0
3,1,0
4,0,1


In [11]:
data_with_dummies = data.drop('category', axis=1).join(dummies)
data_with_dummies

,x0,x1,y,a,b
0,1,0.01,-1.5,1,0
1,2,-0.01,0.0,0,1
2,3,2.50,3.6,1,0
3,4,-4.10,1.3,1,0
4,5,0.00,-2.0,0,1


Now we have only numerical columns and so can convert to an ndarray.

## Creating Model Descriptions with `patsy`

`patsy` uses strings like "y ~ x0 + x1" to create design metrics for a linear model. More detail in the book.

In [12]:
import patsy

In [13]:
data = pd.DataFrame({
    'x0': [1, 2, 3, 4, 5],
    'x1': [0.01, -0.01, 2.5, -4.1, 0.],
    'y': [-1.5, 0., 3.6, 1.3, -2.],
})
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,2.50,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [14]:
y, X = patsy.dmatrices('y ~ x0 + x1', data)
y

DesignMatrix with shape (5, 1)
     y
  -1.5
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

In [15]:
X

DesignMatrix with shape (5, 3)
  Intercept  x0     x1
          1   1   0.01
          1   2  -0.01
          1   3   2.50
          1   4  -4.10
          1   5   0.00
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'x1' (column 2)

These `DesignMatrix` objects are numpy ndarray with a bit more meta-data.

In [16]:
np.asarray(y)

array([[-1.5],
       [ 0. ],
       [ 3.6],
       [ 1.3],
       [-2. ]])

Because these objects are instances of sub-classes of ndarray, they can be passed directly into algorithms like `numpy.linalg.lstsq`, which performs an Ordinary Least Squares (OLS) regression.

In [17]:
coef, resid, _, _ = np.linalg.lstsq(X, y, rcond=0)
coef

array([[0.00501444],
       [0.11327389],
       [0.20261288]])

The patsy design matrices contain the information on which part of the ndarray means what in the model.

In [18]:
coef = pd.Series(coef.squeeze(), index=X.design_info.column_names)
coef

Intercept    0.005014
x0           0.113274
x1           0.202613
dtype: float64

### Data Transformations in Patsy Formulas

You can mix Python code into your patsy formula and patsy will try to call it. The advantage of this I guess if you are going to fit again and again and are lazy about writing functions.

In [19]:
data['np.log(np.abs(x1) + 1)'] = np.log(np.abs(data['x1']) + 1)
data

,x0,x1,y,np.log(np.abs(x1) + 1)
0,1,0.01,-1.5,0.009950
1,2,-0.01,0.0,0.009950
2,3,2.50,3.6,1.252763
3,4,-4.10,1.3,1.629241
4,5,0.00,-2.0,0.000000


And then we could build a model on that, or we can get patsy to do it for us.

In [20]:
y, X = patsy.dmatrices('y ~ x0 + np.log(np.abs(x1) + 1)', data)
X

DesignMatrix with shape (5, 3)
  Intercept  x0  np.log(np.abs(x1) + 1)
          1   1                 0.00995
          1   2                 0.00995
          1   3                 1.25276
          1   4                 1.62924
          1   5                 0.00000
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'np.log(np.abs(x1) + 1)' (column 2)

Patsy has some built-in functions for common operations such as standardizing (to mean 0 and variance 1) and centering (subtracting the mean).  Still not sure why it wouldn't be clearer to just use numpy for these things.

In [21]:
y, X = patsy.dmatrices('y ~ standardize(x0) + center(x1)', data)
X

DesignMatrix with shape (5, 3)
  Intercept  standardize(x0)  center(x1)
          1         -1.41421        0.33
          1         -0.70711        0.31
          1          0.00000        2.82
          1          0.70711       -3.78
          1          1.41421        0.32
  Terms:
    'Intercept' (column 0)
    'standardize(x0)' (column 1)
    'center(x1)' (column 2)

When you transform data in this way, that is, using metrics that depend on all the sample data (such as the `mean` value used for `standardize` and `center`), the distribution as a whole determines the new value, rather than something dependent only on aspects of the sample itself.

A new value like "x0 + x1" depends only on aspects of the sample - it combines two random variables, but not in anyway that depends on the distribution. Similarly, "log(x0)" depends only on x0 - part of a particular sample.

Operations like `standardize` and `center` that depend on "all the data" are called "stateful" in statistics.

Using stateful operations like these also has an impact when generalizing a model - if you hold some data back, the distribution of that new data may not be the same.  Surely we will not use the mean of the originally modeling data, right, but the mean of the new data.  This also carries forward a bit to Traditional ML and Deep Learning.

Just for stats, patsy has a way of accomodating these stateful operations and leveraging new data.

In [22]:
new_data = pd.DataFrame({
    'x0': [6, 7, 8, 9],
    'x1': [3.1, -0.5, 0., 2.3],
    'y': [1, 2, 3, 4]
})
new_data

,x0,x1,y
0,6,3.1,1
1,7,-0.5,2
2,8,0.0,3
3,9,2.3,4


Now we can apply our original `DesignMatrix` to the new data as follows.

In [23]:
new_X = patsy.build_design_matrices([X.design_info], new_data)
new_X

[DesignMatrix with shape (4, 3)
   Intercept  standardize(x0)  center(x1)
           1          2.12132        3.42
           1          2.82843       -0.18
           1          3.53553        0.32
           1          4.24264        2.62
   Terms:
     'Intercept' (column 0)
     'standardize(x0)' (column 1)
     'center(x1)' (column 2)]

Because the "+" symbol in patsy mini-language does not mean addition, if you want to actually add columns, you have to wrap them in an expression like "I(thing1 + thing2)".

In [24]:
y, X = patsy.dmatrices('y ~ I(x0 + x1)', data)
X

DesignMatrix with shape (5, 2)
  Intercept  I(x0 + x1)
          1        1.01
          1        1.99
          1        5.50
          1       -0.10
          1        5.00
  Terms:
    'Intercept' (column 0)
    'I(x0 + x1)' (column 1)

### Categorical Data and patsy

This one is quite beyond me.  Use of non-numerical data in statistical models is based studied in a course in statistics.  It can be treated in many different ways, and that's outside the scope of the book.

When you use non-numeric data with patsy, they are converted to dummy variables by default.  And the author says something that is beyond me - "If there is an intercept, one of the levels will be left out to avoid colinearity.".

In [25]:
data = pd.DataFrame({
    'key1': list('aabbabab'),
    'key2': [0, 1, 0, 1, 0, 1, 0, 0],
    'v1': [1, 2, 3, 4, 5, 6, 7, 8],
    'v2': [-1., 0., 2.5, -0.5, 4., -1.2, 0.2, -1.7]
})
data

,key1,key2,v1,v2
0,a,0,1,-1.0
1,a,1,2,0.0
2,b,0,3,2.5
3,b,1,4,-0.5
4,a,0,5,4.0
5,b,1,6,-1.2
6,a,0,7,0.2
7,b,0,8,-1.7


In [26]:
y, X = patsy.dmatrices('v2 ~ key1', data)
X

DesignMatrix with shape (8, 2)
  Intercept  key1[T.b]
          1          0
          1          0
          1          1
          1          1
          1          0
          1          1
          1          0
          1          1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)

We can see the `DesignMatrix` above only has dummy variables for the `"b"` value of `key1`, a level has been left out.  If we rebuild the design matrix with an explicit intercept of 0, both `"a"` and `"b"` are included.

In [27]:
y, X = patsy.dmatrices('v2 ~ key1 + 0', data)
X

DesignMatrix with shape (8, 2)
  key1[a]  key1[b]
        1        0
        1        0
        0        1
        0        1
        1        0
        0        1
        1        0
        0        1
  Terms:
    'key1' (columns 0:2)

Sometimes numerical columns are truly categorical, such as the case with `key2`. Enclosing them with a C tells patsy to treat them as categorical data.

In [28]:
y, X = patsy.dmatrices('v2 ~ C(key2)', data)
X

DesignMatrix with shape (8, 2)
  Intercept  C(key2)[T.1]
          1             0
          1             1
          1             0
          1             1
          1             0
          1             1
          1             0
          1             0
  Terms:
    'Intercept' (column 0)
    'C(key2)' (column 1)

And this shows what was mean't by colinearity - the categorical data doesn't really have two values as it is already in the dummy format.  So, the last level doesn't carry new information and often can be left out.  If you don't leave it out, in a linear statistics model, you are overproviding the same information, which will have that "colinearity" affect on the model.

When you are using multiple categorical columns at once in a linear statistics model, you can include interaction terms of the form `key1:key2`, which can be used in an analysis of variance (ANOVA) models:

In [29]:
data['key2'] = data['key2'].map({0: 'zero', 1: 'one'})
data

,key1,key2,v1,v2
0,a,zero,1,-1.0
1,a,one,2,0.0
2,b,zero,3,2.5
3,b,one,4,-0.5
4,a,zero,5,4.0
5,b,one,6,-1.2
6,a,zero,7,0.2
7,b,zero,8,-1.7


In [30]:
y, X = patsy.dmatrices('v2 ~ key1 + key2 + key1:key2', data)
X

DesignMatrix with shape (8, 4)
  Intercept  key1[T.b]  key2[T.zero]  key1[T.b]:key2[T.zero]
          1          0             1                       0
          1          0             0                       0
          1          1             1                       1
          1          1             0                       0
          1          0             1                       0
          1          1             0                       0
          1          0             1                       0
          1          1             1                       1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)
    'key1:key2' (column 3)

Patsy provides more capabilities for dealing with categorical data.

## Introduction to `statsmodels`

The chapter starts by saying that `statsmodels` does mostly frequentist statistics, and Bayesian and ML are in scikit-learn and elsewhere.  Let's do some linear algebra to generate data that will fit a given model and that use `statsmodels` to fit it.

In [66]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

Make a normally distributed data with a given mean and variance.

In [33]:
def dnorm(mean, variance, size=1):
    if isinstance(size, int):
        size = size,
    return mean + (np.sqrt(variance) * np.random.randn(*size))

Build an X and y that have a given distribution and parameters

In [59]:
np.random.seed(12345)
N = 100
X = np.c_[dnorm(0, 0.4, size=N), dnorm(0, 0.6, size=N), dnorm(0, 0.2, size=N)]
X[:5]

array([[-0.12946849, -1.21275292,  0.50422488],
       [ 0.30291036, -0.43574176, -0.25417986],
       [-0.32852189, -0.02530153,  0.13835097],
       [-0.35147471, -0.71960511, -0.25821463],
       [ 1.2432688 , -0.37379916, -0.52262905]])

In [60]:
# epsilon perturbs it from the true model produced by np.dot - if we leave off the + eps in building y below, 
# our model should exactly reproduce beta.
eps = dnorm(0, 0.1, size=N)
beta = [0.1, 0.3, 0.5]
# build y
y = np.dot(X, beta) + eps
y[:5]

array([ 0.42786349, -0.67348041, -0.09087764, -0.48949442, -0.12894109])

A linear model is generally fitted by adding a constant intercept.  Let's do that with `statsmodels`.

In [61]:
X_model = sm.add_constant(X)
X_model[:5]

array([[ 1.        , -0.12946849, -1.21275292,  0.50422488],
       [ 1.        ,  0.30291036, -0.43574176, -0.25417986],
       [ 1.        , -0.32852189, -0.02530153,  0.13835097],
       [ 1.        , -0.35147471, -0.71960511, -0.25821463],
       [ 1.        ,  1.2432688 , -0.37379916, -0.52262905]])

Now let's model and fit with OLS.

In [62]:
model = sm.OLS(y, X)
results = model.fit()
results.params

array([0.17826108, 0.22303962, 0.50095093])

In [63]:
print(results.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.430
Model:                            OLS   Adj. R-squared (uncentered):              0.413
Method:                 Least Squares   F-statistic:                              24.42
Date:                Thu, 05 Aug 2021   Prob (F-statistic):                    7.44e-12
Time:                        17:45:56   Log-Likelihood:                         -34.305
No. Observations:                 100   AIC:                                      74.61
Df Residuals:                      97   BIC:                                      82.42
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

We have generic parameters names x1, x2, and x3.  What if this data is put into a `DataFrame` instead.

In [65]:
data = pd.DataFrame(X, columns=['col1', 'col2', 'col3'])
data['y'] = y
data

,col1,col2,col3,y
0,-0.129468,-1.212753,0.504225,0.427863
1,0.302910,-0.435742,-0.254180,-0.673480
2,-0.328522,-0.025302,0.138351,-0.090878
3,-0.351475,-0.719605,-0.258215,-0.489494
4,1.243269,-0.373799,-0.522629,-0.128941
...,...,...,...,...
95,0.502962,0.705644,-0.100857,0.253019
96,0.074699,-0.790788,0.603616,-0.148563
97,-0.473413,-1.094827,0.603872,0.348516
98,0.369967,1.004348,-0.172917,0.204080


Now we can use the `statsmodels` formula api with `patsy` formulas to do the regression.

In [67]:
results = smf.ols('y ~ col1 + col2 + col3', data=data).fit()
results.params

Intercept    0.033559
col1         0.176149
col2         0.224826
col3         0.514808
dtype: float64

In [68]:
results.tvalues

Intercept    0.952188
col1         3.319754
col2         4.850730
col3         6.303971
dtype: float64

Given new out-of-sample or hold out data, you can compute predicted values using the model parameters.

In [69]:
results.predict(data[:5])

0   -0.002327
1   -0.141904
2    0.041226
3   -0.323070
4   -0.100535
dtype: float64

There are many more models and techniques in `statsmodels` beyond OLS.

### Estimating Time Series processes

Let's simulate some time series data with an autoregressive structure and noise.  In an autoregressize process, the output of a process depends linearly on its own previous values and a stochastic (noise) term that cannot be well predicted.

These are modeled with difference equations (not to be confused with differential equations).

*Theory - still use dnorm() for an epsilon.  Random starting position, known vector and constant for terms of the process, which is what the model will try to determine.*

In [80]:
import random

init_x = 4
values = [init_x, init_x]
N = 1000

b1 = 0.8
b2 = -0.4
noise = dnorm(0, 0.1, N)
for i in range(N):
    x = (b1 * values[-1]) + (b2 * values[-2]) + noise[i]
    values.append(x)

This has an AR(2) structure (the next x value depends on the 2 previous terms) with parameters 0.8 and -0.4.  When you fit an AR model, you may not know how many lags to include, so you can fit with some larger number of lags and see what parameters you get.

In [82]:
MAXLAGS = 5
model = sm.tsa.AR(values)
results = model.fit(MAXLAGS)

The parameters have the intercept first, and then the parameters for each lag.

In [83]:
results.params

array([-0.00959585,  0.78080408, -0.39199698,  0.00129287, -0.03843443,
        0.03701866])

You can see that the model got quite close to the actual parameters.

## Introduction to `scikit-learn`

Classical ML and such. Using as an example a now classic dataset about survival rates on the Titanic.

In [84]:
train = pd.read_csv('pydata-book/datasets/titanic/train.csv')
train[:5]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [85]:
test = pd.read_csv('pydata-book/datasets/titanic/test.csv')
test[:5]

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


Look for missing data.

In [86]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [87]:
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

Fill missing ages with the median value - not the only one, but let's get it done.

In [92]:
imputed_age = train['Age'].median()
train['Age'] = train['Age'].fillna(imputed_age)
test['Age'] = test['Age'].fillna(imputed_age)

Make a new column `is_female` which is sort of a dummy variable for the sex - it will be 0 for men and 1 for women.

In [97]:
train['is_female'] = (train['Sex'] == 'female').astype(int)
test['is_female'] = (test['Sex'] == 'female').astype(int)
train[:5]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,is_female
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0


Now that we've done some data mungling with `pandas`, we will transfer what we need to `numpy` (at the time of 2nd edition, scikit-learn cannot act directly on `pandas` objects). 

In [99]:
predictors = ['Pclass', 'is_female', 'Age']
X_train = train[predictors].values
X_train[:5]

array([[ 3.,  0., 22.],
       [ 1.,  1., 38.],
       [ 3.,  1., 26.],
       [ 1.,  1., 35.],
       [ 3.,  0., 35.]])

In [100]:
X_test = test[predictors].values
X_test[:5]

array([[ 3. ,  0. , 34.5],
       [ 3. ,  1. , 47. ],
       [ 2. ,  0. , 62. ],
       [ 3. ,  0. , 27. ],
       [ 3. ,  1. , 22. ]])

In [101]:
y_train = train['Survived'].values
y_train[:5]

array([0, 1, 1, 1, 0])

We'll use the logistic regression model `scikit-learn`.

In [103]:
from sklearn.linear_model import LogisticRegression

In [106]:
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [107]:
y_predict = model.predict(X_test)
y_predict[:5]

array([0, 0, 0, 0, 1])

In this dataset, the test data does not have a 'Survived' column, and so we are missing ground truth. `scikit-learn` has cross validation built into some models so you can calculate hyperparameters.

In [109]:
from sklearn.linear_model import LogisticRegressionCV

In [111]:
model_cv = LogisticRegressionCV(Cs=10)
model_cv.fit(X_train, y_train)

LogisticRegressionCV()

In [112]:
model_cv.predict(X_test)[:5]

array([0, 0, 0, 0, 1])